## Step 0: Understand the Data
Get the info of errors and missing values

In [1]:
import pandas as pd
import numpy as np
s = pd.read_csv("Public_School_Characteristics_2022-23.csv")
s_b = s.copy()
s.shape

C:\Users\mktmo\AppData\Local\Temp\ipykernel_22212\1985055591.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  s = pd.read_csv("Public_School_Characteristics_2022-23.csv")


(101390, 77)

### Pattern for each column
TOTAL = TOTMENROL + TOTFENROL
TOTAL = PK + KG + G01 + G02 + G03 + G04 + G05
      + G06 + G07 + G08
      + G09 + G10 + G11 + G12
      + UG + AE
      
TOTAL = AM + AS + HI + BL + WH + HP + TR

Lunch:
TOTFRL = FRELCH + REDLCH

STUTERATIO = TOTAL / FTE

Elementary Schools:
G09 + G10 + G11 + G12 ≈ 0

High Schools:
PK + KG + G01–G08 ≈ 0

AMALM + AMALF = AM

ASALM + ASALF = AS

BLALM + BLALF = BL

HPALM + HPALF = HP

HIALM + HIALF = HI

TRALM + TRALF = TR

WHALM + WHALF = WH

### Missing Value info

In [2]:
special_values = [-1, -2, -9, "M", "N"]
s = s.replace(r'^\s*$', np.nan, regex=True) # Find any string that is completely empty ('') or only spaces (' '), replace it with np.nan

rows_with_special = s.isin(special_values).any(axis=1).sum()
print(rows_with_special)
rows_with_missing_value = s.isna().any(axis=1).sum()
print(rows_with_missing_value)
total_missing = s.isna().sum().sum()
print(total_missing)


62134
101390
1352278


## Step 1: Data Cleaning
### 1.1 Handling missing 0's for Students Each Grade

In [3]:

cols = s.loc[:, 'PK':'AE'].columns  # all columns from PK to AE

# 1️⃣ Row-wise sum of PK to AE (ignores NaN by default)
row_sum = s[cols].sum(axis=1)

# 2️⃣ Find rows where sum equals TOTAL
mask = row_sum.eq(s['TOTAL'])

# 3️⃣ Fill NaN with 0 only for those rows
s.loc[mask, cols] = s.loc[mask, cols].fillna(0)
print(s.isna().any(axis=1).sum())
print(s.isna().sum().sum())


101244
238819


### 2.2 Removing Redundant Rows and Features

#### Dropping Rows and Features with Too less value

In [4]:
# reference for rows missing value threshold
s_b.loc[49:55,:].isna().mean(axis=1)

49    0.181818
50    0.571429
51    0.571429
52    0.571429
53    0.571429
54    0.571429
55    0.571429
dtype: float64

In [5]:
s = s[s.isna().mean(axis=1) <= 0.55]
print(s.isna().any(axis=1).sum())
print(s.isna().sum().sum())
print(s.isna().sum().sort_values(ascending=False))

99577
164969
LSTREET2      99215
LZIP4         41866
FTE            2565
STUTERATIO     1814
HPALM           941
              ...  
NMCNTY            0
TOTFRL            0
FRELCH            0
REDLCH            0
LONCOD            0
Length: 77, dtype: int64


In [9]:
s = s.loc[:, s.isna().mean() <= 0.40]
print(s.isna().any(axis=1).sum())
print(s.isna().sum().sum())
print(s.isin(special_values).any(axis=1).sum())

5403
23888
60467


In [10]:
s.to_csv("Deliverable2.csv", index=False)

### 2.3 Imputate Continuous Values

### 2.4 Imputate Discrete Values

### 2.5 Handling Edge Cases

### 2.6 Feature Engineering
The features added through the imputation

### 2.7 Dropping Constant Features and Duplicates

### 2.8 Data Shape After Cleaning

## Step 3: Normalization and Transformation